In [14]:
# This script is used for interacting with OpenLCA.
# Codes are adapted from Julian Rickert @GreenDelta by Ao Chen @ETH.
# Connect to IPC server in OpenLCA software before running. (Port: 8080)
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('notebook', {'highlight_selected_word': {
    'highlight_across_all_cells': True,
    'only_cells_in_scroll': False,
    'delay': 500,
    'code_cells_only': True,
}})


import pandas as pd
import olca
import uuid
import math
from datetime import datetime

from matplotlib import pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import rcParams
import matplotlib.patches as mpatches
import seaborn as sns

params = {'mathtext.default': 'regular' }

client = olca.Client(8080)
client

In [35]:
# Get all process dataset in database
process_descriptor = client.get_descriptors(olca.Process)
process_list = []
id_list = []
location_list = []

for process in process_descriptor:
    process_list.append(process.name)
    id_list.append(process.id)
    location_list.append(process.location)
processes_df = pd.DataFrame(list(zip(process_list,
                                   id_list, location_list)),
                               columns=['name', 'id', 'location'])
# processes_df

,name,id,location
0,"heat production, heavy fuel oil, at industrial...",4f582f90-d0ae-3552-be43-70c730e82af0,RoW
1,"containerboard production, fluting medium, rec...",fdffa34a-17fa-3676-9941-0a8a2bc5f5ac,RoW
2,"market for wood chips, dry, measured as dry ma...",35ca5293-f067-326c-ae45-920013c734c7,RER
3,"treatment of waste glass, unsanitary landfill,...",ab082ee7-7469-3a1f-8e8f-f71328b9e842,GLO
4,"heat and power co-generation, natural gas, con...",3c7c0ba3-acd4-3e19-be88-8fff9dc87788,SE
...,...,...,...
19123,"steel production, electric, chromium steel 18/...",39b4f8c5-a396-3e00-b8c1-db951800e388,RoW
19124,"market for glued laminated timber, PUR-glue | ...",c8a2e84f-6e92-3caa-ab1f-388427902eec,CH
19125,"electricity production, hard coal | electricit...",ea622ed5-24a9-3c50-adbb-6f8eb7465bff,US-HICC
19126,"land already in use, annual cropland to forest...",8a8f8368-b831-3525-a257-065cacf08853,BR-TO


In [36]:
# Search specific process databases with key word(s)
# Note that the search is blankspace-sensitive
search_process_df = processes_df[processes_df['name'].str.contains(r'high voltage, production mix(?!$)')] # TODO: change keywords
search_process_df.reset_index(drop=True, inplace=True)

# search_process_df

,name,id,location
0,"electricity, high voltage, production mix | el...",9e9e785d-fa84-34a0-8e2d-c43d37166a8e,CA-SK
1,"electricity, high voltage, production mix | el...",8fd2d8ca-2eb5-361b-8e29-ec4aafcd081d,NL
2,"electricity, high voltage, production mix | el...",becdeb83-db53-3fca-8818-1ed18cacdba6,US-TRE
3,"electricity, high voltage, production mix | el...",6d09283d-34c4-3939-85b7-849819c941d0,CN-SD
4,"electricity, high voltage, production mix | el...",8ec2d93a-b7d4-35d1-9059-a29e4f93b720,TH
...,...,...,...
109,"electricity, high voltage, production mix | el...",dc1c6a45-324e-3e60-8123-ef2aaf7e6001,CN-SC
110,"electricity, high voltage, production mix | el...",1d3a4b99-50d6-3c30-9ece-7c10d2c3d938,AU
111,"electricity, high voltage, production mix | el...",3380a306-abf9-32c1-817d-6ddf33ba49e7,US-FRCC
112,"electricity, high voltage, production mix | el...",fad3093c-f1df-313f-a0ae-4386d9c837b3,SK


In [ ]:
# Get all flow dataset in database
flow_descriptor = client.get_descriptors(olca.Flow)
flow_list = []
id_list = []

for flow in flow_descriptor:
    flow_list.append(flow.name)
    id_list.append(flow.id)
flows_df = pd.DataFrame(list(zip(flow_list,
                                   id_list)),
                               columns=['name', 'id'])
# flows_df

In [ ]:
# Search specific flow databases with key word(s)
# Note that the search is blankspace-sensitive
search_flow_df = flows_df[flows_df['name'].str.contains(r'oil(?!$)')] # TODO: change keywords
search_flow_df.reset_index(drop=True, inplace=True)

# search_flow_df

In [81]:
# Create product systems for selected processes
search_list_processID = list(search_process_df['id'])
product_system_ids = []
product_system_names = []
product_system_locations = pd.Series(search_process_df['location'])

for i in range(len(search_list_processID)):
# for i in range(2): # for test
    print('Creating product system of: '+search_process_df['name'][i]+' | '+product_system_locations[i])
    product_system = client.create_product_system(search_list_processID[i], default_providers='prefer', preferred_type='UNIT_PROCESS') #TODO: preferred_type can be changed to "SYSTEM_PROCESS" if needed
    product_system_ids.append(product_system.id)
    ps = client.find(olca.ProductSystem, search_process_df['name'][i])
    product_system_names.append(ps.name)
    print('Remaining number of product systems to be created: '+str(len(search_list_processID)-i-1))
    # print('Remaining number of product systems to be created: '+str(2-i-1)) # for test

product_system_df = pd.DataFrame(list(zip(product_system_names, product_system_ids)), 
                    columns=['product_system_name', 'product_system_id'])

product_system_df.assign(product_system_location=product_system_locations)

Creating product system of: electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | CA-SK
Remaining number of product systems to be created: 1
Creating product system of: electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | NL
Remaining number of product systems to be created: 0


,product_system_name,product_system_id,product_system_location
0,"electricity, high voltage, production mix | el...",3b8f9ea4-3c76-44a0-9c5c-2e31cf4bb429,CA-SK
1,"electricity, high voltage, production mix | el...",1a21b9f1-ece7-4537-bb2f-3a391f556da0,NL


In [82]:
setup = olca.CalculationSetup()
setup.calculation_type = olca.CalculationType.CONTRIBUTION_ANALYSIS # TODO: Can choose SIMPLE_CALCULATION, CONTRIBUTION_ANALYSIS, UPSTREAM_ANALYSIS, REGIONALIZED_CALCULATION, MONTE_CARLO_SIMULATION
setup.impact_method = client.find(olca.ImpactMethod, 'IPCC 2013') # TODO: choose impact method accordingly

# amount is the amount of the functional unit (fu) of the system that
# should be used in the calculation; unit, flow property, etc. of the fu
# can be also defined; by default openLCA will take the settings of the
# reference flow of the product system
setup.amount = 1.0

In [109]:
for ps in range(len(product_system_names)):
# for ps in range(2): # for test
    
    print('Working on '+product_system_names[ps]+' | '+product_system_locations[ps]+' | '+product_system_ids[ps])

    setup.product_system = client.get(olca.ProductSystem, uid=product_system_ids[ps])

    print(setup.product_system.id)

    calc_result = client.calculate(setup)

    client.excel_export(calc_result, 'calc_result/'+product_system_locations[ps]+'_'+'calc_result.xlsx')

    print('Remaining number of processes to be calculated: '+str(len(product_system_names)-ps-1))

Working on electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | CA-SK | 3b8f9ea4-3c76-44a0-9c5c-2e31cf4bb429
3b8f9ea4-3c76-44a0-9c5c-2e31cf4bb429
Remaining number of processes to be calculated: 1
Working on electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | NL | 1a21b9f1-ece7-4537-bb2f-3a391f556da0
1a21b9f1-ece7-4537-bb2f-3a391f556da0
Remaining number of processes to be calculated: 0


In [177]:
GWP100a_df = pd.DataFrame()
# for i in range(len(product_system_locations)):
for i in range(2): # for test
    filename_xlsx = 'calc_result/'+product_system_locations[i]+'_calc_result.xlsx'
    # filename_xlsx = 'calc_result/'+product_system_locations[0]+'_calc_result.xlsx' # for test
    results_db = pd.ExcelFile(filename_xlsx)
    impact_df = results_db.parse('Impacts')
    setup_df = results_db.parse('Calculation setup')

    new_header = impact_df.iloc[0] #grab the first row for the header
    impact_df = impact_df[1:] #take the data less the header row
    impact_df.columns = new_header #set the header row as the df header

    GWP100a_series = pd.Series(impact_df.iloc[2, 2:])
    GWP100a_series['Process'] = search_process_df['name'][i]
    GWP100a_series['location_extended'] = setup_df.iloc[3, 2]

    GWP100a_df[product_system_locations[i]] = GWP100a_series

GWP100a_df.T.to_excel('GWP100a_summary.xlsx')


/Users/ocean/opt/miniconda3/envs/suscons/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
